In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print("===== TensorFlow GPU Verification =====\n")

# 1. TensorFlow version
print("TensorFlow Version:", tf.__version__)

# 2. List all devices detected by TensorFlow
print("\nAvailable Devices:")
print(device_lib.list_local_devices())

# 3. Check if a GPU is detected
gpus = tf.config.list_physical_devices('GPU')
print("\nTensorFlow GPUs:", gpus)

# 4. Check GPU details if available
if gpus:
    print("\nGPU is detected correctly! 🎉")
    try:
        print("GPU Name:", tf.config.experimental.get_device_details(gpus[0])["device_name"])
    except:
        print("GPU details not available through TensorFlow API.")
else:
    print("\n❌ No GPU detected. Something is wrong.")

# 5. Run a small tensor operation to confirm GPU compute works
print("\nRunning a small computation test...\n")

import time
start = time.time()

# Create large tensor and do matrix multiply
a = tf.random.normal((5000, 5000))
b = tf.random.normal((5000, 5000))
c = tf.matmul(a, b)

end = time.time()
print("Computation finished in:", end - start, "seconds")
print("If this is fast (<1 sec), GPU is being used!")


===== TensorFlow GPU Verification =====

TensorFlow Version: 2.10.0

Available Devices:
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7235903917814531063
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1730412544
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3373724093692546807
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2050, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]

TensorFlow GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

GPU is detected correctly! 🎉
GPU Name: NVIDIA GeForce RTX 2050

Running a small computation test...

Computation finished in: 2.0599331855773926 seconds
If this is fast (<1 sec), GPU is being used!


In [1]:
# Run this cell (may take a few minutes). If packages already installed, pip will skip or upgrade.
!pip install --upgrade pip
!pip install numpy pandas scipy matplotlib opencv-python librosa soundfile scikit-learn tensorflow==2.10.0 flwr==1.5.0


In [9]:
# Imports & config
import os
import glob
import math
import json
from collections import defaultdict
import numpy as np
import pandas as pd
import cv2
import librosa
from typing import Tuple

# Paths (adjust if your dataset root is different)
DATA_ROOT = "StressID Dataset"   # relative path in your repo
VIDEOS_DIR = os.path.join(DATA_ROOT, "Videos")
AUDIO_DIR = os.path.join(DATA_ROOT, "Audio")
PHYSIO_DIR = os.path.join(DATA_ROOT, "Physiological")
LABELS_CSV = r"D:\final_fl\StressID Dataset\labels.csv"

OUT_DIR = "processed"
SESSIONS_OUT = os.path.join(OUT_DIR, "sessions")
CLIENTS_OUT = os.path.join(OUT_DIR, "clients")
LOGS_DIR = "logs"
os.makedirs(SESSIONS_OUT, exist_ok=True)
os.makedirs(CLIENTS_OUT, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)

# Processing settings (chosen for your RTX 2050)
VIDEO_FPS_SAMPLE = 5.0   # fps to sample
VIDEO_SIZE = (48, 48)    # width, height
GRAY = True
AUDIO_SR = 16000
MFCC_N_MFCC = 40
ECG_WINDOW_S = 1.0
ECG_TARGET_LEN = 128

NUM_CLIENTS = 10


In [10]:
# Utility functions
def sample_video_frames(video_path: str,
                        target_size: Tuple[int,int]=VIDEO_SIZE,
                        frames_per_second: float = VIDEO_FPS_SAMPLE,
                        gray: bool = GRAY):
    """Sample frames from a video file and return (frames, timestamps, fps, frame_count)."""
    if not os.path.exists(video_path):
        raise FileNotFoundError(video_path)
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video: {video_path}")
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    sample_interval = max(1, int(round(fps / frames_per_second)))
    frames = []
    timestamps = []
    idx = 0
    success, frame = cap.read()
    while success:
        if idx % sample_interval == 0:
            ts = idx / fps
            frame_resized = cv2.resize(frame, target_size, interpolation=cv2.INTER_AREA)
            if gray:
                frame_resized = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
                frame_resized = np.expand_dims(frame_resized, axis=-1)  # H,W,1
            frames.append(frame_resized)
            timestamps.append(ts)
        idx += 1
        success, frame = cap.read()
    cap.release()
    if len(frames) == 0:
        return np.zeros((0, target_size[1], target_size[0], 1), dtype=np.float32), [], fps, frame_count
    frames = np.array(frames).astype(np.float32) / 255.0
    return frames, timestamps, fps, frame_count

def load_audio(audio_path: str, sr: int = AUDIO_SR):
    """Load audio, return (waveform, sr)."""
    if not os.path.exists(audio_path):
        raise FileNotFoundError(audio_path)
    y, orig_sr = librosa.load(audio_path, sr=None, mono=True)
    if orig_sr != sr:
        y = librosa.resample(y, orig_sr, sr)
    return y, sr

def extract_mfcc(y, sr, n_mfcc=MFCC_N_MFCC, hop_length=160, win_length=400):
    """Return MFCC as (T, n_mfcc)"""
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length,
                                n_fft=512, win_length=win_length)
    return mfcc.T

def load_ecg_txt(path: str) -> pd.DataFrame:
    """
    Flexible reader for physiological txt/csv files.
    Tries several separators; returns DataFrame with at least 'ecg' and, if available, 'time' columns.
    """
    import pandas as pd
    if not os.path.exists(path):
        raise FileNotFoundError(path)
    # Try to read with pandas with heuristics
    try:
        # Try with automatic separator detection
        df = pd.read_csv(path, sep=None, engine='python', header=None)
    except Exception:
        # fallback to whitespace
        df = pd.read_csv(path, delim_whitespace=True, header=None)
    # If one column: assume samples only (no time)
    if df.shape[1] == 1:
        df.columns = ['ecg']
        df['time'] = np.arange(len(df))  # indices; user may have to convert using sample rate
    elif df.shape[1] >= 2:
        # assume first two are (time, value) or (value, something)
        # try to guess which is time: if first col values steadily increase -> it's time
        col0 = df.iloc[:,0].values
        col1 = df.iloc[:,1].values
        if np.all(np.diff(col0) >= 0):
            df = df.iloc[:, :2]
            df.columns = ['time', 'ecg']
        elif np.all(np.diff(col1) >= 0):
            df = df.iloc[:, :2]
            df.columns = ['ecg', 'time']
            df = df.rename(columns={'ecg':'time', 'time':'ecg'})  # swap if mistaken
        else:
            # default: treat first column as ecg, create time index
            df = df.iloc[:, :2]
            df.columns = ['ecg', 'aux']
            df['time'] = np.arange(len(df))
    else:
        raise ValueError("Unable to parse ECG file structure")
    # Ensure numeric
    df['ecg'] = pd.to_numeric(df['ecg'], errors='coerce').fillna(0.0)
    if 'time' in df.columns:
        df['time'] = pd.to_numeric(df['time'], errors='coerce').fillna(np.arange(len(df)))
    return df[['time','ecg']]

def ecg_window_for_time(ecg_df: pd.DataFrame, frame_ts: float, window_s: float = ECG_WINDOW_S, target_len: int = ECG_TARGET_LEN):
    """Return vector of length target_len for ECG centered at frame_ts (seconds)."""
    if 'time' not in ecg_df.columns:
        arr = ecg_df['ecg'].values.astype(np.float32)
    else:
        # select rows with time within [ts - w/2, ts + w/2]
        start = frame_ts - window_s/2.0
        end = frame_ts + window_s/2.0
        window = ecg_df[(ecg_df['time'] >= start) & (ecg_df['time'] <= end)]
        arr = window['ecg'].values.astype(np.float32)
    # pad/truncate to target_len
    if len(arr) == 0:
        arr = np.zeros((target_len,), dtype=np.float32)
    elif len(arr) < target_len:
        pad = np.zeros((target_len - len(arr),), dtype=np.float32)
        arr = np.concatenate([arr, pad])
    else:
        arr = arr[:target_len]
    return arr


In [14]:
# Build LABEL_MAP from your labels.csv format:
# Columns: ['subject/task', 'binary-stress', 'affect3-class']

labels_df = pd.read_csv(LABELS_CSV)

# Clean column names
labels_df.columns = [c.strip().lower() for c in labels_df.columns]

# Now we expect:
# 'subject/task' → session key like '2ea4_Counting1'
# 'binary-stress' → 0 or 1 label

LABEL_MAP = {}

for _, row in labels_df.iterrows():
    session_key = row['subject/task']        # e.g., "2ea4_Counting1"
    stress_label = int(row['binary-stress']) # 0 or 1
    LABEL_MAP[session_key] = stress_label

print("Label entries found:", len(LABEL_MAP))
print("Sample entries:", list(LABEL_MAP.items())[:10])



Label entries found: 700
Sample entries: [('2ea4_Breathing', 0), ('2ea4_Counting1', 1), ('2ea4_Counting2', 1), ('2ea4_Counting3', 1), ('2ea4_Math', 1), ('2ea4_Reading', 0), ('2ea4_Relax', 0), ('2ea4_Speaking', 1), ('2ea4_Stroop', 0), ('2ea4_Video1', 0)]


In [16]:
# ============================
# CELL 5: Build LABEL_MAP
# ============================

import pandas as pd

# Read labels.csv (make sure path is correct)
labels_df = pd.read_csv(LABELS_CSV)

# Clean column names
labels_df.columns = [c.strip().lower() for c in labels_df.columns]

# Expected columns:
# 'subject/task', 'binary-stress', 'affect3-class'

LABEL_MAP = {}

for _, row in labels_df.iterrows():
    session_key = str(row['subject/task']).strip()        # e.g., "2ea4_Counting1"
    stress_label = int(row['binary-stress'])              # 0 or 1
    LABEL_MAP[session_key] = stress_label

print("Label entries found:", len(LABEL_MAP))
print("Sample entries:", list(LABEL_MAP.items())[:10])


Label entries found: 700
Sample entries: [('2ea4_Breathing', 0), ('2ea4_Counting1', 1), ('2ea4_Counting2', 1), ('2ea4_Counting3', 1), ('2ea4_Math', 1), ('2ea4_Reading', 0), ('2ea4_Relax', 0), ('2ea4_Speaking', 1), ('2ea4_Stroop', 0), ('2ea4_Video1', 0)]


In [17]:
# =====================================
# CELL 6: Preprocess all sessions
# =====================================

import os
import numpy as np
import pandas as pd
import cv2
import librosa

# Ensure output directory exists
os.makedirs(SESSIONS_OUT, exist_ok=True)

print("Starting preprocessing...")

video_subjects = sorted([d for d in os.listdir(VIDEOS_DIR) if os.path.isdir(os.path.join(VIDEOS_DIR, d))])
print("Found subjects in Videos folder:", len(video_subjects))


for subj in video_subjects:
    subj_vpath = os.path.join(VIDEOS_DIR, subj)
    tasks = sorted(os.listdir(subj_vpath))

    for video_file in tasks:
        session_key = os.path.splitext(video_file)[0]  # e.g. "2ea4_Counting1"

        print(f"\n=== Processing session: {session_key} ===")

        video_path = os.path.join(subj_vpath, video_file)

        # ----- VIDEO -----
        try:
            frames, timestamps, fps, frame_count = sample_video_frames(video_path)
        except Exception as e:
            print("Video error:", e)
            continue

        # ----- AUDIO -----
        audio_mfcc = None
        audio_path = os.path.join(AUDIO_DIR, subj, session_key + ".wav")
        if os.path.exists(audio_path):
            try:
                y, sr = load_audio(audio_path)
                audio_mfcc = extract_mfcc(y, sr)
            except:
                audio_mfcc = None
        else:
            audio_mfcc = None

        # ----- PHYSIOLOGICAL (ECG) -----
        ecg_df = None
        phys_base = os.path.join(PHYSIO_DIR, subj, session_key)

        # Physiological files may be:
        #   StressID Dataset/Physiological/<subject>/<subject_task>
        #   OR text file
        if os.path.exists(phys_base):
            try:
                ecg_df = load_ecg_txt(phys_base)
            except:
                ecg_df = None
        else:
            for ext in [".txt", ".csv"]:
                if os.path.exists(phys_base + ext):
                    try:
                        ecg_df = load_ecg_txt(phys_base + ext)
                    except:
                        ecg_df = None
                    break

        # ----- ALIGN AUDIO + ECG PER FRAME -----
        per_frame_mfcc = []
        per_frame_ecg = []

        for ts in timestamps:
            # ---- MFCC alignment ----
            if audio_mfcc is not None:
                hop_sec = 160 / AUDIO_SR  # ~10ms hop
                idx = int(round(ts / hop_sec))
                if 0 <= idx < audio_mfcc.shape[0]:
                    per_frame_mfcc.append(audio_mfcc[idx])
                else:
                    per_frame_mfcc.append(np.zeros((MFCC_N_MFCC,), dtype=np.float32))
            else:
                per_frame_mfcc.append(np.zeros((MFCC_N_MFCC,), dtype=np.float32))

            # ---- ECG alignment ----
            if ecg_df is not None:
                ecg_vec = ecg_window_for_time(ecg_df, ts)
                per_frame_ecg.append(ecg_vec)
            else:
                per_frame_ecg.append(np.zeros((ECG_TARGET_LEN,), dtype=np.float32))

        # Convert to np.array
        np_frames = frames
        np_mfcc = np.array(per_frame_mfcc, dtype=np.float32)
        np_ecg = np.array(per_frame_ecg, dtype=np.float32)

        # ----- LABEL -----
        label = LABEL_MAP.get(session_key, -1)  # -1 if not in csv

        # ----- SAVE -----
        out_path = os.path.join(SESSIONS_OUT, f"{session_key}.npz")
        np.savez_compressed(
            out_path,
            frames=np_frames,
            mfcc=np_mfcc,
            ecg=np_ecg,
            timestamps=np.array(timestamps, dtype=np.float32),
            label=np.int32(label),
            meta=np.array([subj], dtype=object)
        )

        print(f"Saved → {out_path}")

print("\n=== Preprocessing completed! ===")


Starting preprocessing...
Found subjects in Videos folder: 53

=== Processing session: .DS_Store ===
Video error: Cannot open video: StressID Dataset\Videos\2ea4\.DS_Store

=== Processing session: 2ea4_Baseline ===
Saved → processed\sessions\2ea4_Baseline.npz

=== Processing session: 2ea4_Breathing ===
Saved → processed\sessions\2ea4_Breathing.npz

=== Processing session: 2ea4_Counting1 ===
Saved → processed\sessions\2ea4_Counting1.npz

=== Processing session: 2ea4_Counting2 ===
Saved → processed\sessions\2ea4_Counting2.npz

=== Processing session: 2ea4_Counting3 ===
Saved → processed\sessions\2ea4_Counting3.npz

=== Processing session: 2ea4_Math ===
Saved → processed\sessions\2ea4_Math.npz

=== Processing session: 2ea4_Reading ===
Saved → processed\sessions\2ea4_Reading.npz

=== Processing session: 2ea4_Relax ===
Saved → processed\sessions\2ea4_Relax.npz

=== Processing session: 2ea4_Speaking ===
Saved → processed\sessions\2ea4_Speaking.npz

=== Processing session: 2ea4_Stroop ===
Sav

In [20]:
# =====================================
# CELL 7: Build 10 clients (ERROR-FREE)
# =====================================

import os
import numpy as np
from collections import defaultdict
import glob
import math

os.makedirs(CLIENTS_OUT, exist_ok=True)

print("Building clients from processed session files...")

# Load session .npz files
session_files = sorted(glob.glob(os.path.join(SESSIONS_OUT, "*.npz")))
print("Total session files found:", len(session_files))

# Group by subject
subject_map = defaultdict(list)

for sess_path in session_files:
    try:
        d = np.load(sess_path, allow_pickle=True)
        subj = str(d["meta"][0])   # subject name stored during preprocessing
        subject_map[subj].append(sess_path)
    except Exception as e:
        print("Warning: Skipping", sess_path, "Error:", e)

subjects = sorted(subject_map.keys())
print("Total unique subjects:", len(subjects))
print("Sample subjects:", subjects[:10])

# Divide subjects into NUM_CLIENTS equal parts
num_subjects = len(subjects)
subjects_per_client = math.ceil(num_subjects / NUM_CLIENTS)

client_splits = []
for i in range(NUM_CLIENTS):
    start = i * subjects_per_client
    end = (i + 1) * subjects_per_client
    chunk = subjects[start:end]
    if chunk:
        client_splits.append(chunk)

print(f"\nCreated {len(client_splits)} client groups.")

# Build client datasets
for client_id, subject_list in enumerate(client_splits):
    print(f"\n=== Building Client {client_id} ===")
    print("Subjects:", subject_list)

    client_frames = []
    client_mfcc = []
    client_ecg = []
    client_labels = []
    client_session_paths = []

    for subj in subject_list:
        for sess_file in subject_map[subj]:
            d = np.load(sess_file, allow_pickle=True)
            client_frames.append(d["frames"])
            client_mfcc.append(d["mfcc"])
            client_ecg.append(d["ecg"])
            client_labels.append(int(d["label"]))
            client_session_paths.append(sess_file)

    out_path = os.path.join(CLIENTS_OUT, f"client_{client_id}.npz")

    np.savez_compressed(
        out_path,
        session_files=np.array(client_session_paths, dtype=object),
        frames=np.array(client_frames, dtype=object),
        mfcc=np.array(client_mfcc, dtype=object),
        ecg=np.array(client_ecg, dtype=object),
        labels=np.array(client_labels, dtype=np.int32)
    )

    print(f"Saved Client {client_id} → {out_path}")
    print(f"  Total sessions: {len(client_session_paths)}")

print("\n=== Client Building Complete! ===")


Building clients from processed session files...
Total session files found: 629
Total unique subjects: 53
Sample subjects: ['2ea4', '2hpu', '2z7d', '45lx', '4e8r', '4woj', '5f7t', '6g6y', '6k5f', '71i5']

Created 9 client groups.

=== Building Client 0 ===
Subjects: ['2ea4', '2hpu', '2z7d', '45lx', '4e8r', '4woj']
Saved Client 0 → processed\clients\client_0.npz
  Total sessions: 72

=== Building Client 1 ===
Subjects: ['5f7t', '6g6y', '6k5f', '71i5', '7h5u', '8g4y']
Saved Client 1 → processed\clients\client_1.npz
  Total sessions: 71

=== Building Client 2 ===
Subjects: ['8i4i', '9j3o', '9t6n', '9txq', 'a1k9', 'b2l8']
Saved Client 2 → processed\clients\client_2.npz
  Total sessions: 72

=== Building Client 3 ===
Subjects: ['b9w0', 'bfl5', 'c3m7', 'chdf', 'ctzy', 'cxj0']
Saved Client 3 → processed\clients\client_3.npz
  Total sessions: 72

=== Building Client 4 ===
Subjects: ['d4n6', 'e5p4', 'f6q3', 'g7r2', 'g9j5', 'h7j3']
Saved Client 4 → processed\clients\client_4.npz
  Total sessions